# Modeling

The configured `Model` in the forge will provide predefined properties of a set of Types if may contain.

In [1]:
from kgforge.core import KnowledgeGraphForge

In [2]:
import getpass

In [3]:
token = getpass.getpass()

In [4]:
forge = KnowledgeGraphForge("../../configurations/demo-forge-nexus.yml", token=token)

# Imports

In [5]:
import json

In [6]:
from kgforge.core import Resource

In [7]:
from kgforge.specializations.resources import Dataset

## Prefixes

Prefixes are namespaces that are used to put Resource properties within a context.

In [8]:
forge.prefixes()

Used prefixes:
 nsg     https://neuroshapes.org/                   
 owl     http://www.w3.org/2002/07/owl#             
 prov    http://www.w3.org/ns/prov#                 
 rdf     http://www.w3.org/1999/02/22-rdf-syntax-ns#
 rdfs    http://www.w3.org/2000/01/rdf-schema#      
 schema  http://schema.org/                         
 skos    http://www.w3.org/2004/02/skos/core#       
 xsd     http://www.w3.org/2001/XMLSchema#          


## Types
The `type` property of a Resource can be associated to the available types in the Model. These types have a pre-defined set of properties.

In [9]:
forge.types()

Managed entity types:
   - Activity
   - Association
   - Building
   - Employee
   - Organization
   - Person
   - PostalAddress


## Templates
The template will provide a set of properties for the givent type that is recomended to be used when creating Resources.

### showing the properties of a type + getting the template of a Mapping for a type

In [10]:
forge.template("Person")

{
    id: ""
    type: Person
    address:
    {
        type: PostalAddress
        postalCode:
        [
            ""
            0
        ]
        streetAddress: ""
    }
    birthDate: 9999-12-31
    deathDate: 9999-12-31
    familyName: ""
    gender:
    [
        female
        male
    ]
    givenName: ""
    name: ""
}


In [11]:
forge.template("Person", only_required=True)

{
    id: ""
    type: Person
    birthDate: 9999-12-31
    familyName: ""
    gender:
    [
        female
        male
    ]
    givenName: ""
    name: ""
}


In [12]:
forge.template("Association")

{
    id: ""
    type: Association
    agent:
    {
        id: ""
        type: Person
        address:
        {
            type: PostalAddress
            postalCode:
            [
                ""
                0
            ]
            streetAddress: ""
        }
        birthDate: 9999-12-31
        deathDate: 9999-12-31
        familyName: ""
        gender:
        [
            female
            male
        ]
        givenName: ""
    }
    name: ""
    parentOrganization:
    {
        id: ""
        type: Organization
    }
}


### creating (a) Resource instance(s)

#### manually (JSON)

In [13]:
forge.template("Person", output="json", only_required=True)

{
    "id": "",
    "type": "Person",
    "address": {
        "type": "PostalAddress",
        "postalCode": [
            "",
            0
        ],
        "streetAddress": ""
    },
    "birthDate": "9999-12-31",
    "familyName": "",
    "deathDate": "9999-12-31",
    "familyName": "",
    "gender": [
        "female",
        "male"
    ],
    "givenName": "",
    "name": ""
}


In [14]:
data = {
    "type": "Person",
    "name": "Jane Doe", 
    "gender": "female"
}

In [15]:
resource_json = forge.from_json(data)

In [16]:
print(resource_json)

{
    type: Person
    gender: female
    name: Jane Doe
}


#### programmatically (Dict)

In [17]:
template = forge.template("Person", output="dict", only_required=True)

In [18]:
template["givenName"] = "Jane"
template["familyName"] = "Doe"

In [19]:
resource_dict = forge.from_json(template)

In [20]:
print(resource_dict)

{
    id: ""
    type: Person
    address:
    {
        type: PostalAddress
        postalCode:
        [
            ""
            0
        ]
        streetAddress: ""
    }
    birthDate: 9999-12-31
    deathDate: 9999-12-31
    familyName: ""
    gender:
    [
        female
        male
    ]
    givenName: ""
    name: Jane
}


## Validation
It is possible to verify that a Resource is compliant with the suggested type schema available in the Model.

In [21]:
jane = Resource(id=forge.format("identifier", "person", "jdoe"), 
                type="Person", givenName="Jane", familyName="Doe", gender="female", birthDate="1985-04-20")

In [22]:
john = Resource(id=forge.format("identifier", "person", "jsmith"), 
                type="Person", givenName="John", familyName="Smith", gender="male", birthDate="1990-04-10")

In [23]:
persons = [jane, john]

In [24]:
forge.validate(persons)

<count> 2
<action> _validate_many
<succeeded> True


In [25]:
jane._last_action

Action(error=None, message=None, operation='_validate_many', succeeded=True)

In [26]:
jane._validated

True

### automatic status update

In [27]:
jane.email = "jane.doe@epfl.ch"

In [28]:
jane._validated

False

### lazy actions handling

In [29]:
! cat ../../data/persons.csv

type,name,distribution
Person,Marie Curie,../../data/scientists-database/marie_curie.txt
Person,Albert Einstein,../../data/scientists-database/albert_einstein.txt


In [30]:
distribution = forge.attach("../../data/persons.csv")

In [31]:
jane = Resource(
    id=forge.format("identifier", "person", "jdoe"), 
    type="Person", 
    givenName="Jane",
    familyName="Doe",
    gender="female", 
    birthDate="1985-04-20",
    distribution=distribution)

In [32]:
forge.validate(jane)

<action> _validate_one
<succeeded> False
<error> ValidationError: resource has lazy actions which need to be executed before


In [33]:
forge.validate(jane, execute_actions_before=True)

<action> _validate_one
<succeeded> True


### error handling

Validating a Resource of type _Person_ that doesn't have an _id_ will fail since _id_ is mandatory.

In [34]:
resource = Resource(type="Person", givenName="John", familyName="Smith", gender="male", birthDate="1990-04-10")

In [35]:
forge.validate(resource)

<action> _validate_one
<succeeded> False
<error> ValidationError: 
Validation Report
Conforms: False
Results (2):
Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:datatype xsd:date ; sh:maxCount Literal("1", datatype=xsd:integer) ; sh:minCount Literal("1", datatype=xsd:integer) ; sh:path schema:birthDate ]
	Focus Node: [ <http://schema.org/familyName> Literal("Smith") ; <http://schema.org/gender> Literal("male") ; <http://schema.org/givenName> Literal("John") ; rdf:type <http://schema.org/Person> ]
	Result Path: schema:birthDate
Constraint Violation in NodeKindConstraintComponent (http://www.w3.org/ns/shacl#NodeKindConstraintComponent):
	Severity: sh:Violation
	Source Shape: this:PersonShape
	Focus Node: [ <http://schema.org/familyName> Literal("Smith") ; <http://schema.org/gender> Literal("male") ; <http://schema.org/givenName> Literal("John") ; rdf:type <http://schema.org/Person> 

In [36]:
resource._last_action

Action(error='ValidationError', message='\nValidation Report\nConforms: False\nResults (2):\nConstraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):\n\tSeverity: sh:Violation\n\tSource Shape: [ sh:datatype xsd:date ; sh:maxCount Literal("1", datatype=xsd:integer) ; sh:minCount Literal("1", datatype=xsd:integer) ; sh:path schema:birthDate ]\n\tFocus Node: [ <http://schema.org/familyName> Literal("Smith") ; <http://schema.org/gender> Literal("male") ; <http://schema.org/givenName> Literal("John") ; rdf:type <http://schema.org/Person> ]\n\tResult Path: schema:birthDate\nConstraint Violation in NodeKindConstraintComponent (http://www.w3.org/ns/shacl#NodeKindConstraintComponent):\n\tSeverity: sh:Violation\n\tSource Shape: this:PersonShape\n\tFocus Node: [ <http://schema.org/familyName> Literal("Smith") ; <http://schema.org/gender> Literal("male") ; <http://schema.org/givenName> Literal("John") ; rdf:type <http://schema.org/Person> ]\n\tV

In [38]:
resource._validated

False